In [9]:
from eeg_research.system.bids_selector  import BidsArchitecture
root = "/data2/Projects/eeg_fmri_natview/derivatives"
architecture = BidsArchitecture(root = root,)

In [10]:
architecture.select(subject=["01","02"])['subject'].unique()

array(['01', '02'], dtype=object)

In [3]:
a = ['a','b','caca','popo']
'caca' in a

True

In [4]:
type(architecture.database['subject'])

pandas.core.series.Series

In [ ]:
import pandas as pd
def is_numerical(dataframe: pd.DataFrame, column_name: str):
    return all(dataframe[column_name].apply(lambda string: string.isdigit()))



is_numerical(architecture.database, 'session')


True

In [33]:
import numpy as np
lower = (2 <= architecture.database['subject'].astype(int))
higher = (architecture.database['subject'].astype(int) < 9)
type(lower & higher)


pandas.core.series.Series

In [32]:
min(architecture.database['subject'].astype(int))

1

In [2]:
import numpy as np
task = []
for file in root.rglob('*'):
    print(file)
    file_parts = architecture.parse_filename(file)
    task.append(file_parts['task'])

task = np.unique(task)
print(task)


/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BVA_GR_CB_BK_NOV2024/sub-18_ses-01_task-tp_run-02_desc-GdCb_eeg.edf
/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BVA_GR_CB_BK_NOV2024/sub-04_ses-01_task-peer_run-01_desc-GdCbBk_eeg.edf
/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BVA_GR_CB_BK_NOV2024/sub-08_ses-02_task-dmh_run-02_eeg-preproc_GD_CB_BK.edf
/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BVA_GR_CB_BK_NOV2024/sub-09_ses-01_task-tp_run-01_eeg-preproc_GD_CB.edf
/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BVA_GR_CB_BK_NOV2024/sub-03_ses-01_task-monkey1_run-01_desc-GdCb_eeg.edf
/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BVA_GR_CB_BK_NOV2024/sub-17_ses-03_task-dme_run-02_desc-GdCbBk_eeg.edf
/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BVA_GR_CB_BK_NOV2024/sub-21_ses-01_task-rest_run-01_desc-GdCbBk_eeg.edf
/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BVA_GR_CB_BK_NOV2024/sub-03_ses-01_task-tp_run-02_desc-GdCb_eeg.edf
/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BVA_GR_CB_BK_NOV2024/sub-06_ses-02_task-monkey2_ru

In [8]:
file.name

'sub-18_ses-01_task-tp_run-02_desc-GdCb_eeg.edf'

In [7]:
architecture.parse_filename('/data2/Projects/eeg_fmri_natview/derivatives/sub-01/ses-01/eeg/sub-01_ses-01_task-rest_run-01_desc-customGfp_eeg.pkl')

{'root': PosixPath('/data2/Projects/eeg_fmri_natview/derivatives'),
 'subject': '01',
 'session': '01',
 'datatype': 'eeg',
 'task': 'rest',
 'run': '01',
 'acquisition': None,
 'description': 'customGfp',
 'suffix': 'eeg',
 'extension': '.pkl',
 'filename': PosixPath('/data2/Projects/eeg_fmri_natview/derivatives/sub-01/ses-01/eeg/sub-01_ses-01_task-rest_run-01_desc-customGfp_eeg.pkl')}

In [ ]:
file_parts = architecture.parse_filename('/data2/Projects/eeg_fmri_natview/derivatives/sub-01/ses-01/eeg/sub-01_ses-01_task-rest_run-01_desc-customGfp_eeg.pkl')
test = architecture._initiate_database_dict(placeholder=[])
for key, value in file_parts.items():
    test[key].append(value)

root
 /data2/Projects/eeg_fmri_natview/derivatives
subject
 01
session
 01
datatype
 eeg
task
 rest
run
 01
acquisition
 None
description
 customGfp
suffix
 eeg
extension
 .pkl
filename
 /data2/Projects/eeg_fmri_natview/derivatives/sub-01/ses-01/eeg/sub-01_ses-01_task-rest_run-01_desc-customGfp_eeg.pkl


In [18]:
None.copy()

AttributeError: 'NoneType' object has no attribute 'copy'

In [10]:
test

{'root': [PosixPath('/data2/Projects/eeg_fmri_natview/derivatives'),
  '01',
  '01',
  'eeg',
  'rest',
  '01',
  None,
  'customGfp',
  'eeg',
  '.pkl',
  PosixPath('/data2/Projects/eeg_fmri_natview/derivatives/sub-01/ses-01/eeg/sub-01_ses-01_task-rest_run-01_desc-customGfp_eeg.pkl')],
 'subject': [PosixPath('/data2/Projects/eeg_fmri_natview/derivatives'),
  '01',
  '01',
  'eeg',
  'rest',
  '01',
  None,
  'customGfp',
  'eeg',
  '.pkl',
  PosixPath('/data2/Projects/eeg_fmri_natview/derivatives/sub-01/ses-01/eeg/sub-01_ses-01_task-rest_run-01_desc-customGfp_eeg.pkl')],
 'session': [PosixPath('/data2/Projects/eeg_fmri_natview/derivatives'),
  '01',
  '01',
  'eeg',
  'rest',
  '01',
  None,
  'customGfp',
  'eeg',
  '.pkl',
  PosixPath('/data2/Projects/eeg_fmri_natview/derivatives/sub-01/ses-01/eeg/sub-01_ses-01_task-rest_run-01_desc-customGfp_eeg.pkl')],
 'datatype': [PosixPath('/data2/Projects/eeg_fmri_natview/derivatives'),
  '01',
  '01',
  'eeg',
  'rest',
  '01',
  None,
  'cus

In [17]:
print([id(lst) for lst in test.values()])

[140040774596096, 140040774596096, 140040774596096, 140040774596096, 140040774596096, 140040774596096, 140040774596096, 140040774596096, 140040774596096, 140040774596096, 140040774596096]


In [ ]:
import numpy as np

np.isin(selector.files_dataframe['subject'].values,['01','05','06'])

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,

In [ ]:
selector.files_dataframe